# Extract junction loops from [RNA representative structures](http://rna.bgsu.edu/rna3dhub/nrlist)
Since junction loop motifs are not available by [RNA 3D motif Atlas](http://rna.bgsu.edu/rna3dhub/motifs), download non-redundant RNA representative structures and extract junction loops manually

In [1]:
import os, sys, shutil
import pathlib
import glob as glob
import numpy as np
import json
import wget
import time
import warnings

In [2]:
def make_dir(output_path):
    """
    create directory
    """
    
    if os.path.isdir(output_path):
        print(">remove directory: {}".format(output_path))
        shutil.rmtree(output_path)
        
    pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [3]:
def download_csv(output_path, file, url_loop):
    """
    Download csv containing loop motif information
    
    Parameters
    ----------
    output_path : str
        File path pointing to the output directory 
    file : str
        CSV file in full path that contains non-redundant representative RNA information
    url_loop : str
        URL to download RNA loop strucutres (e.g. hairpin, internal, and junction) using BGSU API (https://www.bgsu.edu/research/rna/APIs.html)
    
    Returns
    -------
    """
    
    with open(file, "r") as f:
        for l in f.readlines():
            pdb  = l.split(',')[1].split('|')[0].replace('"', '')            
            csv  = os.path.join(output_path, "dump", pdb + "_loop.csv")                        
            _url = os.path.join(url_loop, pdb)
            
            try:
                # same pdb could be present with different chain id
                if os.path.isfile(csv):
                    #print("{} already found".format(os.path.basename(csv)))
                    pass
                else:
                    wget.download(_url, out=csv, bar=None)
            except:
                warnings.warn("Could not download {}\n{}".format(pdb, _url))
            
            time.sleep(1)  # avoid api traffic

In [4]:
def loadfile(file):
    """
    Load CSV file and get juction loop IDs
    
    Parameters
    ----------
    file : str
        CSV file in full path that stores loop IDs (e.g. hairpin: HP, internal: IL, junction: J3)
        
    Returns
    -------
    loop_ids : list of str
        List of junction loop IDs
    """
    
    loop_ids = []
    with open(file, 'r') as f:
        for l in f.readlines():
            
            # only extract junction loops
            if l.startswith('"J3_'):
                loop_id = l.split(',')[0].strip('"')
                loop_ids.append(loop_id)
            
        if len(loop_ids) == 0:
            print("no junction loops found: {}".format(os.path.basename(file)))
                
    return loop_ids

In [5]:
def download_cif(output_path, release_version, loop_ids, url_coord):
    """
    Download RNA coordinates using BGSU APIs (https://www.bgsu.edu/research/rna/APIs.html)
    
    Parameters
    ----------
    output_path : str
        Full path pointing to the output cif file
    release_version : str
        Release version of Representative sets of RNA 3D structures (http://rna.bgsu.edu/rna3dhub/nrlist)
    loop_ids : list of str
        Junciton loop id (e.g. J3_6SVS_002)
    url : str
        API url to download RNA coordinates
        
    Returns
    -------
    """
    
    for loop_id in loop_ids:
        _url = url_coord + loop_id
        cif = os.path.join(output_path, "dump", loop_id + ".cif")
        
        # check duplicate motif entry
        if os.path.exists(cif):
            warnings.warn("{} already exists. Duplicate motif entry.".format(loop_id))
        
        try:
            wget.download(_url, out=cif, bar=None)
            extract_model(output_path, release_version, cif) 
        except:
            warnings.warn("Could not download {}\n{}".format(loop_id, _url))

In [6]:
def extract_model(output_path, release_version, cif):
    """
    Extract the first model from the cif file. The downloaded cif file contains two models. Model 1 is the RNA motif of interset and model 2 is the neighboring structures of model 1. 
    
    Parameters
    ----------
    output_path : str
        Full path pointing to the output cif file
    release_version : str
        Release version of Representative sets of RNA 3D structures (http://rna.bgsu.edu/rna3dhub/nrlist)
    cif : str
        Downloaded cif file in full path
    
    Returns
    -------
    """
    arr = []
    count = 0
    with open(cif, "r") as f:
        for l in f.readlines():
            _l = l.strip('\n').split()[0]

            if _l.startswith("#"):
                count += 1
            if count == 2:    
                break
                
            arr.append(l)
    
    ofile = os.path.join(output_path, os.path.basename(cif))
    with open(ofile, "w") as wf:
        for l in arr:
            wf.write(l)

In [7]:
if __name__ == "__main__":
    
    url_coord = "http://rna.bgsu.edu/rna3dhub/rest/getCoordinates?coord="
    url_loop = "http://rna.bgsu.edu/rna3dhub/loops/download"
    base_path = os.path.dirname(os.path.abspath("__file__")).strip('notebooks')
    release_version = "nrlist_3.245_4.0A"    
    
    # create output directory
    output_path = os.path.join(base_path, "data", release_version, "JunctionLoop")
    make_dir(output_path)
    make_dir(os.path.join(output_path, "dump"))
    
    # download junction loops via RNA representative structures
    print(">download csv")
    file = os.path.join(base_path, "data", release_version + ".csv")
    download_csv(output_path, file, url_loop)    

    # extract loop junctions for each entry
    print(">download cif")
    files = glob.glob(os.path.join(output_path, "dump", "*_loop.csv"))
    for file in files:
        loop_ids = loadfile(file)
        download_cif(output_path, release_version, loop_ids, url_coord)

>download csv


/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1786194711.py:32: UserWarning: Could not download 5GAH
http://rna.bgsu.edu/rna3dhub/loops/download/5GAH
  warnings.warn("Could not download {}\n{}".format(pdb, _url))
/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1786194711.py:32: UserWarning: Could not download 1HMH
http://rna.bgsu.edu/rna3dhub/loops/download/1HMH
  warnings.warn("Could not download {}\n{}".format(pdb, _url))


>download cif
no junction loops found: 7OX9_loop.csv
no junction loops found: 3BBM_loop.csv
no junction loops found: 7OX8_loop.csv
no junction loops found: 4XNR_loop.csv
no junction loops found: 5HR6_loop.csv
no junction loops found: 5HR7_loop.csv
no junction loops found: 5NDH_loop.csv
no junction loops found: 5NDI_loop.csv
no junction loops found: 1ET4_loop.csv
no junction loops found: 7KHA_loop.csv
no junction loops found: 1WZ2_loop.csv
no junction loops found: 5LTA_loop.csv
no junction loops found: 7D7V_loop.csv
no junction loops found: 4NGD_loop.csv
no junction loops found: 7BPV_loop.csv
no junction loops found: 7D7W_loop.csv
no junction loops found: 5EEU_loop.csv
no junction loops found: 5KK5_loop.csv
no junction loops found: 6ZWW_loop.csv
no junction loops found: 7JJU_loop.csv
no junction loops found: 6O5F_loop.csv
no junction loops found: 5ED2_loop.csv
no junction loops found: 4QVD_loop.csv
no junction loops found: 4RDX_loop.csv
no junction loops found: 3K64_loop.csv
no junction

/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:32: UserWarning: Could not download J3_7P48_001
http://rna.bgsu.edu/rna3dhub/rest/getCoordinates?coord=J3_7P48_001
  warnings.warn("Could not download {}\n{}".format(loop_id, _url))


no junction loops found: 7TJ2_loop.csv
no junction loops found: 6FPQ_loop.csv
no junction loops found: 2O5I_loop.csv
no junction loops found: 2NR0_loop.csv
no junction loops found: 3LA5_loop.csv
no junction loops found: 6GFW_loop.csv
no junction loops found: 1YYK_loop.csv
no junction loops found: 5B2T_loop.csv
no junction loops found: 7N0D_loop.csv
no junction loops found: 2ZUE_loop.csv
no junction loops found: 3IEV_loop.csv
no junction loops found: 4WZJ_loop.csv
no junction loops found: 1YFG_loop.csv
no junction loops found: 6AR5_loop.csv
no junction loops found: 2BGG_loop.csv
no junction loops found: 6RIN_loop.csv
no junction loops found: 5JJI_loop.csv
no junction loops found: 6UGG_loop.csv
no junction loops found: 2Y8W_loop.csv
no junction loops found: 1NBS_loop.csv
no junction loops found: 4BXX_loop.csv
no junction loops found: 6QIT_loop.csv
no junction loops found: 6BSI_loop.csv
no junction loops found: 3S1Q_loop.csv
no junction loops found: 5GJB_loop.csv
no junction loops found: 

#### redo failed download

In [9]:
# csv
pdbs  = ["5GAH", "1HMH"]
for pdb in pdbs:
    csv  = os.path.join(output_path, "dump", pdb + "_loop.csv")                        
    _url = "http://rna.bgsu.edu/rna3dhub/loops/download/{}".format(pdb)
    wget.download(_url, out=csv, bar=None)

In [10]:
# cif

pdbs = ["5GAH", "1HMH", "7P48"]
for pdb in pdbs:
    file = os.path.join(output_path, "dump", "{}_loop.csv".format(pdb))
    loop_ids = loadfile(file)
    download_cif(output_path, release_version, loop_ids, url_coord)

no junction loops found: 1HMH_loop.csv


/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:26: UserWarning: J3_7P48_002 already exists. Duplicate motif entry.
  warnings.warn("{} already exists. Duplicate motif entry.".format(loop_id))
/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:26: UserWarning: J3_7P48_003 already exists. Duplicate motif entry.
  warnings.warn("{} already exists. Duplicate motif entry.".format(loop_id))
/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:26: UserWarning: J3_7P48_004 already exists. Duplicate motif entry.
  warnings.warn("{} already exists. Duplicate motif entry.".format(loop_id))
/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:26: UserWarning: J3_7P48_005 already exists. Duplicate motif entry.
  warnings.warn("{} already exists. Duplicate motif entry.".format(loop_id))
/var/folders/4w/d4mqk6kx5js1ycjth49ps228ljmkky/T/ipykernel_88653/1560134999.py:26: UserWarning: J3_7P48_